In [1]:
# Imports
import pyodbc
import pandas as pd
from faker import Faker
import numpy as np

In [2]:
# Defina os parâmetros de conexão
SQL_CONN_SERVER = '192.168.2.193\\SQLDEV2019'
SQL_DATABASE = 'PARetail'
SQL_USER = 'airflow'
SQL_PASS = 'airflow'

SQL_TABLE_SRC =  '__Cliente'
SQL_TABLE_DST =  'Clientes'

In [7]:
# Crie a string de conexão
#connection_string = f'DRIVER={{SQL Server}};SERVER={SQL_CONN_SERVER};DATABASE={SQL_DATABASE};UID={SQL_USER};PWD={SQL_PASS}'
connection_string = (
            f'DRIVER={{SQL Server}};'
            f'SERVER={SQL_CONN_SERVER};'
            f'DATABASE={SQL_DATABASE};'
            f'UID={SQL_USER};'
            f'PWD={SQL_PASS}'
        )

In [4]:
# Inicializa o Faker
fake = Faker('pt_BR') 

In [25]:
# Estabeleça a conexão
connection = pyodbc.connect(connection_string)

# Clientes

In [37]:
cursor = connection.cursor()

# Execute uma consulta
cursor.execute("SELECT top(30000) [ID_SQL], [TaxId4], [CardName], [Cellular], [E_Mail] FROM [dbo].[__Cliente] WHERE CardName in (select CardName FROM [dbo].[__Cliente] group by CardName having count(*) >  1)")
# cursor.execute(
#     'select [ID_SQL], [TaxId4], [CardName], [Cellular], [E_Mail] FROM [dbo].[__Cliente] c where exists (select 1 from (select [TaxId4], MAX(ID_SQL) MX_ID FROM [dbo].[__Cliente] group by [TaxId4] having count(*) >  1) x where x.TaxId4=c.TaxId4 and x.MX_ID = c.ID_SQL)'
# )

rows = cursor.fetchall()


# Atualiza os dados com valores falsos
for row in rows:
    id = row[0]
    cpf_falso = fake.cpf()
    nome_falso = fake.name()
    celular_falso = fake.cellphone_number()
    email_falso = fake.email()
    cursor.execute(
        'UPDATE [dbo].[__Cliente] SET [TaxId4] = ?, [CardName] = ?, [Cellular] = ?, [E_Mail] = ? WHERE [ID_SQL] = ?',
        (cpf_falso, nome_falso, celular_falso, email_falso, id)
    )
    
# Confirma as alterações
connection.commit()

# Fecha a conexão
cursor.close()
connection.close()


# Vendedores

In [26]:
cursor = connection.cursor()

# Execute uma consulta
cursor.execute("SELECT [codigo],[nome] from Vendedores")
rows = cursor.fetchall()


# Atualiza os dados com valores falsos
for row in rows:
    codigo = row[0]
    primeiro_nome_falso = fake.first_name()
    ultimo_nome_falso = fake.last_name()
    concat_cod_name = str(codigo) + '-' + primeiro_nome_falso 
    cursor.execute(
        'UPDATE [dbo].[Vendedores] SET [nome] = ? WHERE [codigo] = ?',
        (concat_cod_name, codigo)
    )
    # print(f"Codigo: {codigo}, Nome: {concat_cod_name}")
    
# Confirma as alterações
connection.commit()

# Fecha a conexão
cursor.close()
connection.close()